In [10]:
import ee
ee.Authenticate()

True

In [11]:
ee.Initialize(project="dlamb6-2")

In [12]:
dem = ee.Image('USGS/3DEP/10m')

geom = ee.Geometry.Point([-91.0989573,30.3529013])

geom_col = ee.FeatureCollection([geom])

elev = dem.sampleRegions(geom_col).getInfo()
elev['features'][0]['properties']['elevation']

3.220296621322632

In [5]:
import pandas as pd

table = pd.read_csv(r"C:\Users\dlamb6\desktop\GISProgrammingProject\boundary.csv")
table.head()

,col,row,X,Y
0,4871,174,699102.8878,186780.4458
1,4871,174,699102.8878,186780.4458
2,4872,174,699105.8874,186780.4458
3,4870,175,699099.8882,186777.4462
4,4873,174,699108.8870,186780.4458


In [13]:
import arcpy

ra1 = arcpy.Raster(r"C:\Users\dlamb6\desktop\flood_2class.tif")
print( ra1.spatialReference.factoryCode)

32119


In [14]:
import geopandas as gpd
import geopandas
gdf = gpd.GeoDataFrame(table)
gdf.set_geometry( geopandas.points_from_xy(gdf['X'], gdf['Y']), inplace=True, crs=f'EPSG:{ra1.spatialReference.factoryCode}')


In [15]:
gdf.to_file(r"C:\Users\dlamb6\desktop\GISProgrammingProject\boundary_2.shp")

In [16]:
shapefile = r'C:\Users\dlamb6\desktop\GISProgrammingProject\boundary_2.shp'
arcpy.management.AddField(shapefile,'elevation',field_type='FLOAT')

<Result 'C:\\Users\\dlamb6\\Desktop\\GISProgrammingProject\\boundary_2.shp'>

In [17]:
geom_list = []
with arcpy.da.SearchCursor(shapefile,['SHAPE@XY'],spatial_reference=4326) as cursor:
    for row in cursor:
        X,Y = row[0]
        geom = ee.Geometry.Point([X,Y])
        geom_list.append(geom)
geom_col = ee.FeatureCollection(geom_list)
elev = dem.sampleRegions(geom_col).getInfo().get('features')

In [18]:
import arcpy
i = 0
with arcpy.da.UpdateCursor(shapefile,['elevation']) as cursor:
    for row in cursor:
        elevation = elev[i]['properties']['elevation']
        row[0] = elevation
        cursor.updateRow(row)
        i += 1